In [40]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go

In [41]:
file_path = 'output.csv'
df = pd.read_csv(file_path)

# Define dates
df.columns = df.columns.str.strip()  # Strip whitespace
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

company_name = df['Company'].unique().tolist()

df = df.reset_index()  # Move the index into columns
df = df.set_index(['Company', 'Date'])  # Now set 'Company' and 'Date' as the index



In [42]:
df.head()

index   Adj Close       Close        High         Low  \
Company Date                                                                
APPLE   2024-09-06      0  220.577332  220.820007  225.240005  219.770004   
        2024-09-09      1  220.667221  220.910004  221.270004  216.710007   
        2024-09-10      2  219.868103  220.110001  221.479996  216.729996   
        2024-09-11      3  222.415298  222.660004  223.089996  217.889999   
        2024-09-12      4  222.525177  222.770004  223.550003  219.820007   

                          Open    Volume  LSTM Predictions  RF Predictions  
Company Date                                                                
APPLE   2024-09-06  223.949997  48423000        217.669837      221.968604  
        2024-09-09  220.820007  67180000        217.265124      221.819303  
        2024-09-10  218.919998  51591000        216.749693      221.818804  
        2024-09-11  221.460007  44587100        216.145360      220.822303  
        2024-09-12  222.500000  37498200        215.614873      221.378803

In [43]:
company_name

['APPLE', 'GOOGLE', 'MICROSOFT', 'AMAZON', 'NVIDIA', 'META', 'TESLA', 'INTEL']

In [44]:
actual_data = {}
predicted_data_lstm = {}
predicted_data_rf = {}
for company in company_name:
    company_data = df[df.index.get_level_values('Company') == company]
    predicted_data_lstm[company] = company_data['LSTM Predictions'].to_dict()
    predicted_data_rf[company] = company_data['RF Predictions'].to_dict()
    actual_data[company] = company_data['Adj Close'].to_dict()

actual_data['META']


{('META', Timestamp('2024-09-06 00:00:00')): 499.7931823730469,
 ('META', Timestamp('2024-09-09 00:00:00')): 504.3088989257813,
 ('META', Timestamp('2024-09-10 00:00:00')): 504.3088989257813,
 ('META', Timestamp('2024-09-11 00:00:00')): 511.3421630859375,
 ('META', Timestamp('2024-09-12 00:00:00')): 525.0990600585938,
 ('META', Timestamp('2024-09-13 00:00:00')): 524.1199951171875,
 ('META', Timestamp('2024-09-16 00:00:00')): 533.280029296875,
 ('META', Timestamp('2024-09-17 00:00:00')): 536.3200073242188,
 ('META', Timestamp('2024-09-18 00:00:00')): 537.9500122070312,
 ('META', Timestamp('2024-09-19 00:00:00')): 559.0999755859375,
 ('META', Timestamp('2024-09-20 00:00:00')): 561.3499755859375,
 ('META', Timestamp('2024-09-23 00:00:00')): 564.4099731445312,
 ('META', Timestamp('2024-09-24 00:00:00')): 563.3300170898438,
 ('META', Timestamp('2024-09-25 00:00:00')): 568.3099975585938,
 ('META', Timestamp('2024-09-26 00:00:00')): 567.8400268554688,
 ('META', Timestamp('2024-09-27 00:00:00'

In [ ]:
# Functions
def plot_correlation_heatmap(df):
    correlation_matrix = pd.DataFrame(df).corr()
    fig = go.Figure(
        data=go.Heatmap(
            z=correlation_matrix.values,
            x=correlation_matrix.columns,
            y=correlation_matrix.index,
            colorscale='Viridis',
            colorbar=dict(title="Correlation")
        )
    )
    fig.update_layout(
        title="Stock Correlation Heatmap",
        xaxis_title="Stocks",
        yaxis_title="Stocks",
        template="plotly_dark"
    )
    return fig

def calculate_risk_metrics(price_data, risk_free_rate=0.01):
    returns = price_data.pct_change().dropna()
    mean_return = returns.mean()
    volatility = returns.std()
    sharpe_ratio = (mean_return - risk_free_rate) / volatility
    downside_returns = returns[returns < 0]
    downside_volatility = downside_returns.std()
    sortino_ratio = (mean_return - risk_free_rate) / downside_volatility if downside_volatility != 0 else np.nan
    return sharpe_ratio, sortino_ratio, volatility

def calculate_cumulative_returns(price_data):
    return (price_data.pct_change().fillna(0) + 1).cumprod()


def calculate_portfolio_performance(actual_data, weights):
    price_data = [list(actual_data[ticker].values()) for ticker in company_name]
    portfolio_data = np.dot(np.array(price_data).T, weights)
    return portfolio_data

# Initialize app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = dbc.Container([
    html.H1("Stock Price Prediction Dashboard", className="text-center my-4"),

    # Overview Section
    html.Div([
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader("Latest Stock Price"),
                    dbc.CardBody([
                        html.H4(id="latest-stock-price", className="card-title"),
                        html.P("Latest price of the selected stock", className="card-text")
                    ])
                ], color="primary", inverse=True),
            ], width=4),
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader("Portfolio Key Metrics"),
                    dbc.CardBody([
                        html.H4(id="portfolio-key-metrics", className="card-title"),
                        html.P("Sharpe Ratio, Sortino Ratio, etc.", className="card-text")
                    ])
                ], color="success", inverse=True),
            ], width=4),
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader("Current Portfolio Performance"),
                    dbc.CardBody([
                        html.H4(id="portfolio-performance", className="card-title"),
                        html.P("Total value and return %", className="card-text")
                    ])
                ], color="info", inverse=True),
            ], width=4),
        ], className="mb-4")
    ]),

    # Dropdown for company selection
    dbc.Row([
        dbc.Col([
            html.Label("Select Company:"),
            dcc.Dropdown(
                id="company-dropdown",
                options=[{'label': name, 'value': ticker} for ticker, name in zip(company_name, ['Apple', 'Google', 'Microsoft', 'Amazon', 'NVIDIA', 'META'])],
                value='APPLE',
                clearable=False
            ),
        ], width=4),
        dbc.Col([
            html.Label("Select Prediction Model:"),
            dcc.RadioItems(
                id="model-toggle",
                options=[{'label': 'LSTM', 'value': 'LSTM'}, {'label': 'Random Forest', 'value': 'Random Forest'}],
                value='LSTM',
                inline=True
            ),
        ], width=4),
    ], className="my-3"),

    # Time-series graph
    dcc.Graph(id="time-series-graph", style={"height": "500px"}),

    # Metrics display
    html.Div(id="performance-metrics", className="mt-4"),

    # Risk Metrics
    html.Div(id="risk-metrics", className="mt-4"),

    # Correlation Matrix
    dcc.Graph(id="correlation-heatmap", style={"height": "500px"}),

    html.Hr(),

    # Portfolio Optimization
    html.H3("Portfolio Optimization", className="text-center my-4"),
    
    dbc.Row([
        dbc.Col([
            html.Label("Risk Tolerance:"),
            dcc.Slider(
                id="risk-tolerance-slider",
                min=0.0, max=1.0, step=0.1, value=0.5,
                marks={i/10: f"{i/10}" for i in range(0, 11)}
            )
        ], width=6),
        dbc.Col([
            html.Label("Optimization Criteria:"),
            dcc.RadioItems(
                id="optimization-criteria",
                options=[{'label': 'Maximize Sharpe Ratio', 'value': 'sharpe'}, {'label': 'Minimize Risk', 'value': 'risk'}],
                value='sharpe',
                inline=True
            )
        ], width=6),
    ], className="my-3"),

    # Allocation chart
    dcc.Graph(id="allocation-chart", style={"height": "500px"}),

    # Portfolio Comparison
    dcc.Graph(id="portfolio-comparison", style={"height": "500px"}),

    # Cumulative Returns
    dcc.Graph(id="cumulative-returns", style={"height": "500px"})
])

# Callback for updates
@app.callback(
    [Output("latest-stock-price", "children"),
     Output("portfolio-key-metrics", "children"),
     Output("portfolio-performance", "children"),
     Output("time-series-graph", "figure"),
     Output("performance-metrics", "children"),
     Output("risk-metrics", "children"),
     Output("correlation-heatmap", "figure"),
     Output("allocation-chart", "figure"),
     Output("portfolio-comparison", "figure"),
     Output("cumulative-returns", "figure")],
    [Input("company-dropdown", "value"),
     Input("model-toggle", "value"),
     Input("risk-tolerance-slider", "value"),
     Input("optimization-criteria", "value")]
)
def update_dashboard(company, model, risk_tolerance, optimization_criteria):
    # Prepare data
    historical = actual_data[company]
    predicted = predicted_data_lstm[company] if model == 'LSTM' else predicted_data_rf[company]

    # Latest Stock Price
    company_data = {key: value for key, value in historical.items() if key[0] == company}
    latest_price = company_data[max(company_data, key=lambda x: x[1])]  # Max based on date
    latest_stock_price = f"${latest_price:.2f}"

    # Portfolio Key Metrics
    sharpe_ratio, sortino_ratio, volatility = calculate_risk_metrics(pd.Series(historical))
    portfolio_key_metrics = f"Sharpe Ratio: {sharpe_ratio:.2f}, Sortino Ratio: {sortino_ratio:.2f}, Volatility: {volatility:.2%}"

    # Current Portfolio Performance
    portfolio_weights = np.random.rand(len(company_name)) * risk_tolerance
    portfolio_weights /= np.sum(portfolio_weights)
    portfolio_value = np.dot([actual_data[ticker][max(actual_data[ticker])] for ticker in company_name], portfolio_weights)
    portfolio_performance = f"Total Value: ${portfolio_value:.2f}, Return: {((portfolio_value / np.sum(portfolio_weights)) - 1):.2%}"

    # Time Series Graph
    dates = [key[1] for key in historical.keys()]  
    values = list(historical.values())
    sorted_data = sorted(zip(dates, values))
    dates, values = zip(*sorted_data)

    # Extract predicted values corresponding to the dates
    predicted_values = [predicted[('APPLE', date)] for date in dates if ('APPLE', date) in predicted]

    # Plot the time series
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=values, mode='lines', name='Historical'))
    fig.add_trace(go.Scatter(x=dates, y=predicted_values, mode='lines', name='Predicted'))
    fig.update_layout(
        title=f"Stock Prices for {company} ({model} Model)",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )

    # Performance Metrics
    aligned_keys = historical.keys() & predicted.keys()  # Intersection of keys
    historical_values = np.array([historical[key] for key in aligned_keys])
    predicted_values = np.array([predicted[key] for key in aligned_keys])

    mae = np.mean(np.abs(historical_values - predicted_values))
    rmse = np.sqrt(np.mean((historical_values - predicted_values) ** 2))
    metrics_div = dbc.Row([
        dbc.Col(html.Div(f"MAE: {mae:.2f}", className="text-white bg-primary p-2"), width=3),
        dbc.Col(html.Div(f"RMSE: {rmse:.2f}", className="text-white bg-info p-2"), width=3),
    ])

    # Risk Metrics
    risk_metrics_div = dbc.Row([
        dbc.Col(html.Div(f"Sharpe Ratio: {sharpe_ratio:.2f}", className="text-white bg-success p-2"), width=4),
        dbc.Col(html.Div(f"Sortino Ratio: {sortino_ratio:.2f}", className="text-white bg-warning p-2"), width=4),
        dbc.Col(html.Div(f"Volatility: {volatility:.2f}", className="text-white bg-danger p-2"), width=4),
    ])

    # Correlation Heatmap
    correlation_fig = plot_correlation_heatmap(actual_data)

    # Portfolio Optimization
    allocation_chart = go.Figure(data=[go.Pie(labels=company_name, values=portfolio_weights)])
    allocation_chart.update_layout(title="Portfolio Allocation")

    # Portfolio Comparison
    price_values = [list(actual_data[ticker].values()) for ticker in company_name]
    original_portfolio = np.mean(price_values, axis=0)
    optimized_portfolio = calculate_portfolio_performance(actual_data, portfolio_weights)
    portfolio_comparison_fig = go.Figure()
    portfolio_comparison_fig.add_trace(go.Scatter(x=dates, y=original_portfolio, mode='lines', name="Original Portfolio"))
    portfolio_comparison_fig.add_trace(go.Scatter(x=dates, y=optimized_portfolio, mode='lines', name="Optimized Portfolio"))

    # Cumulative Returns
    cumulative_returns_fig = go.Figure()
    cumulative_returns_fig.add_trace(go.Scatter(x=dates, y=calculate_cumulative_returns(pd.Series(historical)), mode='lines', name="Cumulative Returns"))

    return (latest_stock_price, portfolio_key_metrics, portfolio_performance, fig,
            metrics_div, risk_metrics_div, correlation_fig, allocation_chart,
            portfolio_comparison_fig, cumulative_returns_fig)


if __name__ == "__main__":
    app.run_server(debug=True)


C:\Users\Jaideep\AppData\Local\Temp\ipykernel_23344\1158311839.py:10: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

